<a href="https://colab.research.google.com/github/alyneberriel/pipelines/blob/main/pub_sub_ApacheBeam_base_Flights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade pip

In [ ]:
!pip install apache_beam[interactive]

In [ ]:
pip install apache_beam[gcp]

In [ ]:
import apache_beam as beam
import os
from apache_beam.options.pipeline_options import PipelineOptions

In [ ]:
pip install google-cloud-pubsub

In [ ]:
#SoftWARE que pRoduz mensagens - SIMULADOR DE UM SISTEMA EM STREAMING

import csv
import time
from google.cloud import pubsub_v1
import os

serviceAccount = '/content/bc23-soulcode-1bb1e93aa2c5.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

topico = 'projects/bc23-soulcode/topics/alyneconsumer01'
publisher = pubsub_v1.PublisherClient()

entrada = '/content/voos.csv'


with open(entrada, 'rb') as file:
  next(file)
  for row in file:
    print('Linha enviada')
    publisher.publish(topico,row)
    #time.sleep(1)

In [ ]:
#Para puxar as mensagens do topico 1, mesmo sem fazer as transformações das pipeline - apenas como teste de envio e recebimento

from google.pubsub_v1.types.pubsub import Subscription
import csv
import time
from google.cloud import pubsub_v1
import os

serviceAccount = '/content/bc23-soulcode-1bb1e93aa2c5.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

subscription = 'projects/bc23-soulcode/subscriptions/alynesub01'
subscriber = pubsub_v1.SubscriberClient()

def visualizar_msg(mensagem):
  print(('mensagem:{}'.format(mensagem)))
  mensagem.ack()

subscriber.subscribe(subscription,callback=visualizar_msg)

while True:
  time.sleep(4)

In [ ]:
from apache_beam.io.gcp.bigquery import PCollection
#Modelo Streaming apra dataflow - PIPELINE DE TRANFORMAÇÕES QUE O SIMULADOR DE SISTEMA ENVIOU PARA O PUBSUB1 - NECESSARIO ATIVAR O JOB PARA RODAR A PIPELINE
# #ele busca as informações no SUB1 tranforma o arquivo pegando os dados que queremos e envia para o topico 2
# PEGANDO DO 1 - subscription = 'projects/bc23-soulcode/subscriptions/alynesub01'
# SALVANDO DO 2 - saida = 'projects/bc23-soulcode/topics/alyneconsumer02'
# Modelo para criar nova tabela com dados agregados de duas pipes

import apache_beam as beam
import os 
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam import window

pipeline_options ={
    'project':'bc23-soulcode',
    'runner':'DataflowRunner',
    'region':'southamerica-east1',
    'staging_location':'gs://bases_tratamento/staging/',
    'temp_location':'gs://bases_tratamento/staging/',
    'template_location':'gs://bases_tratamento/models/modelo_streaming',
    'save_main_session':True,
    'streaming':True
}
serviceAccount = '/content/bc23-soulcode-1bb1e93aa2c5.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

pipeline_options = PipelineOptions.from_dictionary(pipeline_options)

p1 = beam.Pipeline(options=pipeline_options)

subscription = 'projects/bc23-soulcode/subscriptions/alynesub01'
saida = 'projects/bc23-soulcode/topics/alyneconsumer02'

class separador(beam.DoFn):
  def process(self,record):
    return [record.decode("utf-8").split(',')]

pcollection_entrada=(
  p1 
  | 'Ler do topico1' >>beam.io.ReadFromPubSub(subscription=subscription)  

)


qte_tempo = (
    pcollection_entrada
    |'Separador do csv' >> beam.ParDo(separador())
    |'Filtrar aero'>> beam.Filter(lambda record: float(record[8])>0)
    |'Agregar as colunas'>> beam.Map(lambda record: (record[4],float(record[8])))
    |'Janela de sliding'>> beam.WindowInto(window.SlidingWindows(15,10))
    |'Construir uma nova tabela por minuto' >> beam.CombinePerKey(sum)
    #|'Exibir o resultado'>> beam.Map(print)
   # |'Gravar o resultado'>> beam.io.WriteToText('voos_minutos.txt')
)

qte_vezes = (
    pcollection_entrada
    |'1Separador do csv' >> beam.Map(lambda record: record.split(','))
    |'1Filtrar aero'>> beam.Filter(lambda record: float(record[8])>0)
    |'1Agregar as colunas'>> beam.Map(lambda record: (record[4],float(record[8])))
    |'1Janela de sliding'>> beam.WindowInto(window.SlidingWindows(15,10))
    |'1Construir uma nova tabela por qtd' >> beam.combiners.Count.PerKey()
    #|'Exibir o resultado'>> beam.Map(print)
    #|'Gravar o resultado'>> beam.io.WriteToText('voos_minutos.txt')
)

tabela = (
    {'Quantidade_minutos':qte_tempo,'Numero_de_atrasos':qte_vezes}
    |'Agrupar as pernas' >> beam.CoGroupByKey()
    |'Converter para PubSub'>> beam.Map(lambda row:(''.join(str(row)).encode('utf-8')))
    |'Gravar o resultado'>> beam.io.WriteToPubSub(saida)
    
)
execucao = p1.run()
execucao.wait_until_finish()

In [ ]:
#Consumo final do pub/sub 2 - SIMULADOR DE UM SISTEMA EM STREAMING BUSCANDO OS DADOS NA ASSINATURA2

from google.pubsub_v1.types.pubsub import Subscription
import csv
import time
from google.cloud import pubsub_v1
import os

serviceAccount = '/content/bc23-soulcode-1bb1e93aa2c5.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

subscription = 'projects/bc23-soulcode/subscriptions/alynesub02'
subscriber = pubsub_v1.SubscriberClient()

def visualizar_msg(mensagem):
  print(('mensagem:{}'.format(mensagem)))
  mensagem.ack()
  time.sleep(2)

subscriber.subscribe(subscription,callback=visualizar_msg)

while True:
  time.sleep(4)